In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Feb 21 06:28:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    52W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os 
import numpy as np
import pandas as pd
import json
import warnings
import logging
import gc
import random
import math
import re
import ast
from tqdm import tqdm
from typing import Optional
from datetime import datetime
import pickle

from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import jaccard_score, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.8 MB/s eta 0:00:00


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
# from rouge_score.rouge_scorer import RougeScorer

warnings.filterwarnings("ignore")

import torch
from torch.utils.data import DataLoader, TensorDataset

from transformers import (
    BartTokenizerFast,
    AdamW
)

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("Using GPU")

else:
    DEVICE = torch.device("cpu")
    print("Using CPU")

foldNum = 0

from transformers import BertTokenizer, BertModel

SOURCE_MAX_LEN = 500
# TARGET_MAX_LEN = 50
# MAX_UTTERANCES = 25

ACOUSTIC_DIM = 768
ACOUSTIC_MAX_LEN = 1000



VISUAL_DIM = 2048
VISUAL_MAX_LEN = 480



import random




LEARNING_RATE = 1e-4


VALID_LEN = 69

# BASE_LEARNING_RATE = 5e-6
# NEW_LEARNING_RATE = 5e-5
# WEIGHT_DECAY = 1e-4

# NUM_BEAMS = 5
# EARLY_STOPPING = True
# NO_REPEAT_NGRAM_SIZE = 3

# EARLY_STOPPING_THRESHOLD = 5

Using GPU


In [ ]:
def set_random_seed(seed: int):
    print("Seed : {}".format(seed))

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.deterministic = True

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# set_random_seed(42)
# a = np.random.randint(0, 1000)

# set_random_seed(123)
set_random_seed(994)
# set_random_seed(12345)

import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint
from torch.nn import CrossEntropyLoss, MSELoss

from typing import Any, Callable, Dict, Iterable, List, Optional, Tuple, Union

from transformers.modeling_utils import PreTrainedModel, unwrap_model

from transformers import (
    BartTokenizer,
    AdamW
)

from transformers.models.bart.configuration_bart import BartConfig

from transformers.models.bart.modeling_bart import (
    BartPretrainedModel,
    BartDecoder,
    BartLearnedPositionalEmbedding,
    BartEncoderLayer,
    shift_tokens_right,
    _make_causal_mask,
    _expand_mask
)

from transformers.modeling_outputs import (
    BaseModelOutput,
    Seq2SeqLMOutput,
    Seq2SeqModelOutput,
    Seq2SeqSequenceClassifierOutput
)

Seed : 994


In [ ]:
# from transformer_encoder import TransformerEncoder

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# bert_tokenizer  = BertTokenizer.from_pretrained("bert-base-cased")
# bert_model = BertModel.from_pretrained("bert-base-cased")
# # bert_model

In [ ]:
BART_INPUTS_DOCSTRING = r"""
    Args:
        input_ids (`torch.LongTensor` of shape `(batch_size, sequence_length)`):
            Indices of input sequence tokens in the vocabulary. Padding will be ignored by default should you provide
            it.
            Indices can be obtained using [`BartTokenizer`]. See [`PreTrainedTokenizer.encode`] and
            [`PreTrainedTokenizer.__call__`] for details.
            [What are input IDs?](../glossary#input-ids)
        attention_mask (`torch.Tensor` of shape `(batch_size, sequence_length)`, *optional*):
            Mask to avoid performing attention on padding token indices. Mask values selected in `[0, 1]`:
            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.
            [What are attention masks?](../glossary#attention-mask)
        decoder_input_ids (`torch.LongTensor` of shape `(batch_size, target_sequence_length)`, *optional*):
            Indices of decoder input sequence tokens in the vocabulary.
            Indices can be obtained using [`BartTokenizer`]. See [`PreTrainedTokenizer.encode`] and
            [`PreTrainedTokenizer.__call__`] for details.
            [What are decoder input IDs?](../glossary#decoder-input-ids)
            Bart uses the `eos_token_id` as the starting token for `decoder_input_ids` generation. If `past_key_values`
            is used, optionally only the last `decoder_input_ids` have to be input (see `past_key_values`).
            For translation and summarization training, `decoder_input_ids` should be provided. If no
            `decoder_input_ids` is provided, the model will create this tensor by shifting the `input_ids` to the right
            for denoising pre-training following the paper.
        decoder_attention_mask (`torch.LongTensor` of shape `(batch_size, target_sequence_length)`, *optional*):
            Default behavior: generate a tensor that ignores pad tokens in `decoder_input_ids`. Causal mask will also
            be used by default.
            If you want to change padding behavior, you should read [`modeling_bart._prepare_decoder_attention_mask`]
            and modify to your needs. See diagram 1 in [the paper](https://arxiv.org/abs/1910.13461) for more
            information on the default strategy.
        head_mask (`torch.Tensor` of shape `(encoder_layers, encoder_attention_heads)`, *optional*):
            Mask to nullify selected heads of the attention modules in the encoder. Mask values selected in `[0, 1]`:
            - 1 indicates the head is **not masked**,
            - 0 indicates the head is **masked**.
        decoder_head_mask (`torch.Tensor` of shape `(decoder_layers, decoder_attention_heads)`, *optional*):
            Mask to nullify selected heads of the attention modules in the decoder. Mask values selected in `[0, 1]`:
            - 1 indicates the head is **not masked**,
            - 0 indicates the head is **masked**.
        cross_attn_head_mask (`torch.Tensor` of shape `(decoder_layers, decoder_attention_heads)`, *optional*):
            Mask to nullify selected heads of the cross-attention modules in the decoder. Mask values selected in `[0,
            1]`:
            - 1 indicates the head is **not masked**,
            - 0 indicates the head is **masked**.
        encoder_outputs (`tuple(tuple(torch.FloatTensor)`, *optional*):
            Tuple consists of (`last_hidden_state`, *optional*: `hidden_states`, *optional*: `attentions`)
            `last_hidden_state` of shape `(batch_size, sequence_length, hidden_size)`, *optional*) is a sequence of
            hidden-states at the output of the last layer of the encoder. Used in the cross-attention of the decoder.
        past_key_values (`tuple(tuple(torch.FloatTensor))`, *optional*, returned when `use_cache=True` is passed or when `config.use_cache=True`):
            Tuple of `tuple(torch.FloatTensor)` of length `config.n_layers`, with each tuple having 2 tensors of shape
            `(batch_size, num_heads, sequence_length, embed_size_per_head)`) and 2 additional tensors of shape
            `(batch_size, num_heads, encoder_sequence_length, embed_size_per_head)`.
            Contains pre-computed hidden-states (key and values in the self-attention blocks and in the cross-attention
            blocks) that can be used (see `past_key_values` input) to speed up sequential decoding.
            If `past_key_values` are used, the user can optionally input only the last `decoder_input_ids` (those that
            don't have their past key value states given to this model) of shape `(batch_size, 1)` instead of all
            `decoder_input_ids` of shape `(batch_size, sequence_length)`. inputs_embeds (`torch.FloatTensor` of shape
            `(batch_size, sequence_length, hidden_size)`, *optional*): Optionally, instead of passing `input_ids` you
            can choose to directly pass an embedded representation. This is useful if you want more control over how to
            convert `input_ids` indices into associated vectors than the model's internal embedding lookup matrix.
        decoder_inputs_embeds (`torch.FloatTensor` of shape `(batch_size, target_sequence_length, hidden_size)`, *optional*):
            Optionally, instead of passing `decoder_input_ids` you can choose to directly pass an embedded
            representation. If `past_key_values` is used, optionally only the last `decoder_inputs_embeds` have to be
            input (see `past_key_values`). This is useful if you want more control over how to convert
            `decoder_input_ids` indices into associated vectors than the model's internal embedding lookup matrix.
            If `decoder_input_ids` and `decoder_inputs_embeds` are both unset, `decoder_inputs_embeds` takes the value
            of `inputs_embeds`.
        use_cache (`bool`, *optional*):
            If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding (see
            `past_key_values`).
        output_attentions (`bool`, *optional*):
            Whether or not to return the attentions tensors of all attention layers. See `attentions` under returned
            tensors for more detail.
        output_hidden_states (`bool`, *optional*):
            Whether or not to return the hidden states of all layers. See `hidden_states` under returned tensors for
            more detail.
        return_dict (`bool`, *optional*):
            Whether or not to return a [`~utils.ModelOutput`] instead of a plain tuple.
"""



In [ ]:
class ContextAwareAttention(nn.Module):

    def __init__(self,
                 dim_model : int,
                 dim_context : int,
                 dropout_rate : Optional[float] = 0.0 ):

        super(ContextAwareAttention, self).__init__()

        self.dim_model = dim_model
        self.dim_context = dim_context
        self.dropout_rate = dropout_rate
        self.attention_layer = nn.MultiheadAttention(embed_dim=self.dim_model,
                                                     num_heads = 1,
                                                     dropout = self.dropout_rate,
                                                     bias = True,
                                                    add_zero_attn=False,
                                                    batch_first=True,
                                                    device=DEVICE   
        )

        self.u_k = nn.Linear(self.dim_context, self.dim_model, bias = False)
        self.w1_k = nn.Linear(self.dim_model, 1, bias=False)
        self.w2_k = nn.Linear(self.dim_model, 1, bias=False)

        self.u_v = nn.Linear(self.dim_context, self.dim_model, bias=False)
        self.w1_v = nn.Linear(self.dim_model, 1, bias = False)
        self.w2_v = nn.Linear(self.dim_model, 1, bias = False)

    def forward(self, q, k, v, context):

        # print("Context shape : ", context.shape)
        # print("Dim context : ", self.dim_context, " : Dim model : ", self.dim_model)
        key_context = self.u_k(context)
        # print("Context shape below key context : ", key_context.shape)
        value_context = self.u_v(context)

        lambda_k = F.sigmoid(self.w1_k(k) + self.w2_k(key_context))
        lambda_v = F.sigmoid(self.w1_v(v) + self.w2_v(value_context))

        k_cap = (1-lambda_k) * k + (lambda_k) * key_context
        v_cap = (1-lambda_v) * v + (lambda_v) * value_context

        attention_output, _ = self.attention_layer(query = q,
                                                   key = k_cap,
                                                   value = v_cap)
  
        return attention_output                                     



In [ ]:
class MAF_acoustic(nn.Module):
    def __init__(self, 
                dim_model,
                dropout_rate):
        super(MAF_acoustic, self).__init__()
        self.dropout_rate = dropout_rate

        self.acoustic_context_transform = nn.Linear(ACOUSTIC_MAX_LEN, SOURCE_MAX_LEN, bias = False)
        # self.visual_context_transform = nn.Linear(VISUAL_MAX_LEN, SOURCE_MAX_LEN, bias = False)

        self.acoustic_context_attention = ContextAwareAttention(dim_model=dim_model,
                                                                dim_context=ACOUSTIC_DIM,
                                                                dropout_rate=dropout_rate)

        # self.visual_context_attention = ContextAwareAttention(dim_model=dim_model,
        #                                                     dim_context=VISUAL_DIM,
        #                                                     dropout_rate=dropout_rate)

        self.acoustic_gate = nn.Linear(2*dim_model, dim_model)
        # self.visual_gate = nn.Linear(2*dim_model, dim_model)
        self.dropout_layer = nn.Dropout(dropout_rate)
        self.final_layer_norm = nn.LayerNorm(dim_model)

    def forward(self,
                text_input,
                acoustic_context):

        # print("Acoustic context shape (A) : ", acoustic_context.shape)        

        acoustic_context = acoustic_context.permute(0,2,1)
        acoustic_context = self.acoustic_context_transform(acoustic_context.float())
        acoustic_context = acoustic_context.permute(0,2,1)

        audio_out = self.acoustic_context_attention(q=text_input,
                                                    k=text_input,
                                                    v=text_input,
                                                    context=acoustic_context)
        # print("Audio out (A) : ", audio_out.shape) 

        # print("Visual context shape : ", visual_context.shape)
        # visual_context = visual_context.permute(0,2,1)
        # visual_context = self.visual_context_transform(visual_context.float())
        # visual_context = visual_context.permute(0,2,1)
        
        # video_out = self.visual_context_attention(q=text_input,
        #                                             k=text_input,
        #                                             v=text_input,
        #                                             context=visual_context)

        # print("Video out shape : ", video_out.shape)
        # print("Text input shape : ", text_input.shape)
        weight_a = F.sigmoid(self.acoustic_gate(torch.cat([text_input, audio_out], dim=-1)))
        # weight_v = F.sigmoid(self.visual_gate(torch.cat([text_input, video_out], dim=-1)))

        # output = self.final_layer_norm(text_input + weight_a * audio_out + weight_v * video_out)

        output = self.final_layer_norm(text_input + weight_a * audio_out)

        return output

In [ ]:
class MAF_visual(nn.Module):
    def __init__(self, 
                dim_model,
                dropout_rate):
        super(MAF_visual, self).__init__()
        self.dropout_rate = dropout_rate

        # self.acoustic_context_transform = nn.Linear(ACOUSTIC_MAX_LEN, SOURCE_MAX_LEN, bias = False)
        self.visual_context_transform = nn.Linear(VISUAL_MAX_LEN, SOURCE_MAX_LEN, bias = False)

        # self.acoustic_context_attention = ContextAwareAttention(dim_model=dim_model,
        #                                                         dim_context=ACOUSTIC_DIM,
        #                                                         dropout_rate=dropout_rate)

        self.visual_context_attention = ContextAwareAttention(dim_model=dim_model,
                                                            dim_context=VISUAL_DIM,
                                                            dropout_rate=dropout_rate)

        # self.acoustic_gate = nn.Linear(2*dim_model, dim_model)
        self.visual_gate = nn.Linear(2*dim_model, dim_model)
        self.dropout_layer = nn.Dropout(dropout_rate)
        self.final_layer_norm = nn.LayerNorm(dim_model)

    def forward(self,
                text_input,
                visual_context):

        # print("Acoustic context shape (A) : ", acoustic_context.shape)        

        # acoustic_context = acoustic_context.permute(0,2,1)
        # acoustic_context = self.acoustic_context_transform(acoustic_context.float())
        # acoustic_context = acoustic_context.permute(0,2,1)

        # audio_out = self.acoustic_context_attention(q=text_input,
        #                                             k=text_input,
        #                                             v=text_input,
        #                                             context=acoustic_context)
        # print("Audio out (A) : ", audio_out.shape) 

        # print("Visual context shape : ", visual_context.shape)
        visual_context = visual_context.permute(0,2,1)
        visual_context = self.visual_context_transform(visual_context.float())
        visual_context = visual_context.permute(0,2,1)
        
        video_out = self.visual_context_attention(q=text_input,
                                                    k=text_input,
                                                    v=text_input,
                                                    context=visual_context)

        # print("Video out shape : ", video_out.shape)
        # print("Text input shape : ", text_input.shape)
        # weight_a = F.sigmoid(self.acoustic_gate(torch.cat([text_input, audio_out], dim=-1)))
        weight_v = F.sigmoid(self.visual_gate(torch.cat([text_input, video_out], dim=-1)))

        # output = self.final_layer_norm(text_input + weight_a * audio_out + weight_v * video_out)

        output = self.final_layer_norm(text_input  + weight_v * video_out)

        return output

In [ ]:
# class CustomTransformerEncoder(torch.nn.Module):
#   def __init__(self, vocab_size, hidden_dim, expand_factor):
#     super(CustomTransformerEncoder, self).__init__()


In [ ]:
# class MultiHeadSelfAttention(torch.nn.Module):
#   def __init__(self, config):
#     super(MultiHeadSelfAttention, self).__init__()

#     self.num_attention_heads = 12
#     self.attention_head_size = config.d_model//12
#     self.embed_dim = config.d_model
#     self.dropout = config.dropout

#     self.query = nn.Linear(config.d_model, config.d_model)
#     self.key = nn.Linear(config.d_model, config.d_model)
#     self.value = nn.Linear(config.d_model, config.d_model)
#     self.out_proj = nn.Linear(config.d_model, config.d_model)

#   def forward(self, hidden_states, attention_mask):
#     query_states = self.query(hidden_states)
#     key_states = self.key(hidden_states)
#     value_states = self.value(hidden_states)

#     batch_size = hidden_states.shape[0]
#     seq_len = hidden_states.shape[1]

#     query_states =  query_states.view(batch_size, seq_len, self.num_attention_heads, self.attention_head_size).transpose(1,2).contiguous()
#     key_states =  key_states.view(batch_size, seq_len, self.num_attention_heads, self.attention_head_size).transpose(1,2).contiguous()
#     value_states = value_states.view(batch_size, seq_len, self.num_attention_heads, self.attention_head_size).transpose(1,2).contiguous()

#     query_states = query_states.view(batch_size*self.num_attention_heads, -1, self.attention_head_size)
#     key_states  = key_states.view(batch_size*self.num_attention_heads, -1, self.attention_head_size)
#     value_states = value_states.view(batch_size*self.num_attention_heads, -1, self.attention_head_size)

#     attn_weights = torch.bmm(query_states, key_states.transpose(1,2))

#     attn_weights = attn_weights.view(batch_size, self.num_attention_heads, attn_weights.shape[1], attn_weights.shape[2])
#     attn_weights = attn_weights/torch.sqrt(torch.tensor(self.attention_head_size))

#     attn_weights_masked = attn_weights.masked_fill(
#         attention_mask[:, None, None, :] == False, -1e16
#     )

#     attn_weights = torch.nn.functional.softmax(attn_weights_masked, dim = -1)

#     attn_weights = torch.nn.functional.dropout(attn_weights, p = self.dropout, training = self.training)

#     attn_weights = attn_weights.view(batch_size*self.num_attention_heads, seq_len, seq_len)


    
#     attn_output = torch.bmm(attn_weights, value_states)

#     attn_output = attn_output.view(batch_size, self.num_attention_heads, seq_len, self.attention_head_size)
#     attn_output = attn_output.transpose(1,2)

#     attn_output = attn_output.reshape(batch_size, seq_len, self.embed_dim)

#     attn_out = self.out_proj(attn_output)

#     return attn_out

In [ ]:
# def gelu(input):
#   input = torch.tensor(input)
#   return input * 0.5 * (1.0 + torch.erf(input/torch.sqrt(torch.tensor(2.0))))

In [ ]:
# class CustomEncoder(torch.nn.Module):
#   def __init__(self, config):
#     super(CustomEncoder, self).__init__()
#     self.embed_dim = config.d_model
#     self.self_attn = MultiHeadSelfAttention(config)
#     self.self_attn_layer_norm = nn.LayerNorm(self.embed_dim)
#     self.dropout = config.dropout
#     # self.activation_fn = torch.nn.ReLU()
#     self.activation_dropout = config.activation_dropout
#     self.fc1 = nn.Linear(self.embed_dim, 2*self.embed_dim)
#     self.fc2 = nn.Linear(2*self.embed_dim, self.embed_dim)
#     self.final_layer_norm = nn.LayerNorm(self.embed_dim)

#   def forward(self, hidden_states, attention_mask):
#    residual = hidden_states
#    hidden_states =  self.self_attn(hidden_states = hidden_states, attention_mask = attention_mask)
#    hidden_states = nn.functional.dropout(hidden_states, p = self.dropout, training = self.training)
#    hidden_states = residual + hidden_states
#    hidden_states = self.self_attn_layer_norm(hidden_states)

#    residual = hidden_states
#    hidden_states = gelu(self.fc1(hidden_states))
#    hidden_states = nn.functional.dropout(hidden_states, p=self.activation_dropout, training = self.training)
#    hidden_states = self.fc2(hidden_states)
#    hidden_states = nn.functional.dropout(hidden_states, p=self.activation_dropout, training = self.training)
#    hidden_states = residual + hidden_states
#    hidden_states = self.final_layer_norm(hidden_states)

#    if hidden_states.dtype == torch.float16 and (
#        torch.isinf(hidden_states).any() or torch.isnan(hidden_states).any()
#    ):
#       clamp_value = torch.finfo(hidden_states.dtype).max = -1000
#       hidden_states = torch.clamp(hidden_states, min = -clamp_value, max = clamp_value)

#    return hidden_states   

In [ ]:
# class ContextEncoder(nn.Module):
#   def __init__(self, config, num_layers):
#     super(ContextEncoder, self).__init__()

#     self.word_embeddings = nn.Embedding(config.vocab_size, config.d_model, padding_idx = config.pad_token_id)
#     self.position_embedding = nn.Embedding(500, config.d_model)
#     self.layer_norm = nn.LayerNorm(config.d_model)
#     self.context_encoder = torch.nn.ModuleList([CustomEncoder(config) for _ in range(num_layers)])
#     self.context_gate = nn.Linear(2*config.d_model, config.d_model)
    

#   def forward(self, hidden_states, context_input_ids, context_attention_mask) :
    
#     seq_len = context_input_ids.shape[1]
#     pos = torch.arange(seq_len, dtype = torch.long)
#     pos = pos.unsqueeze(0).expand_as(context_input_ids)
#     pos = pos.to(device)
#     context_state = self.word_embeddings(context_input_ids) + self.position_embedding(pos)
    
#     for layer in self.context_encoder:
#             context_state = layer(hidden_states = context_state, attention_mask  = context_attention_mask)
    

#     weight_c = F.sigmoid(self.context_gate(torch.cat([hidden_states, context_state], dim=-1)))
#     output = self.layer_norm(hidden_states + weight_c * context_state)
#     # print('transformer context shape :', output.shape)
#     return output



In [ ]:
# class ContextEncoder(nn.Module):
#   def __init__(self, config):
#     super(ContextEncoder, self).__init__()

#     self.bert = bert_model
#     self.layer_norm = nn.LayerNorm(config.d_model)
#     self.context_gate = nn.Linear(2*config.d_model, config.d_model)
    

#   def forward(self, hidden_states, context_input_ids, context_attention_mask) :
    
#     output_encoding = self.bert(context_input_ids, context_attention_mask)

#     context_state = output_encoding['last_hidden_state']
    

#     weight_c = F.sigmoid(self.context_gate(torch.cat([hidden_states, context_state], dim=-1)))
#     output = self.layer_norm(hidden_states + weight_c * context_state)
#     # print('transformer context shape :', output.shape)
#     return output



In [ ]:
class MultiModalBartEncoder(BartPretrainedModel):

    def __init__(self, config: BartConfig, embed_tokens: Optional[nn.Embedding] = None):
        super().__init__(config)

        self.dropout = config.dropout
        self.layerdrop = config.encoder_layerdrop

        embed_dim = config.d_model
        self.padding_idx = config.pad_token_id
        self.max_source_position = config.max_position_embeddings
        self.embed_scale = math.sqrt(embed_dim) if config.scale_embedding else 1.0

        if embed_tokens is not None:
            self.embed_tokens = embed_tokens
        else:
            self.embed_tokens = nn.Embedding(config.vocab_size, embed_dim, self.padding_idx)

        self.embed_positions = BartLearnedPositionalEmbedding(
            config.max_position_embeddings,
            embed_dim
        )        

        
        

        self.layers = nn.ModuleList([BartEncoderLayer(config) for _ in range(config.encoder_layers)])

        self.layernorm_embedding = nn.LayerNorm(embed_dim)

        self.init_weights()
        self.gradient_checkpointing = False

        # self.fusion_at_layer = [4]
        # self.fusion_at_layer = [3, 4]
        # self.fusion_at_layer3 = [3]
        self.fusion_at_layer4 = [4]
        self.fusion_at_layer5 = [2]

        # self.fusion_of_context = [3]
        # self.visual_transformer = TransformerEncoder(d_model = VISUAL_DIM,
        #                                              n_layers = 4,
        #                                              n_heads=8,
        #                                              d_ff=VISUAL_DIM
        #                                              )
        # self.acoustic_transformer = TransformerEncoder(d_model = ACOUSTIC_DIM,
        #                                                n_layers=4,
        #                                                n_heads=2,
        #                                                d_ff=ACOUSTIC_DIM)

        # self.MAF_layer3 = MAF(dim_model=embed_dim,
        #                      dropout_rate=0.2)
        
        self.MAF_layer4 = MAF_acoustic(dim_model=embed_dim,
                             dropout_rate=0.2)
        
        self.MAF_layer5 = MAF_visual(dim_model=embed_dim,
                             dropout_rate=0.2)
        
        # self.context_encoder = ContextEncoder(config)

        # self.classification = nn.Linear(embed_dim, 2)


    def forward(self,
            input_ids = None,
            attention_mask = None,
            # context_input_ids = None,
            # context_attention_mask = None,
            acoustic_input = None,
            visual_input = None,
            head_mask = None,
            inputs_embeds = None,
            output_attentions = None,
            output_hidden_states  = None,
            return_dict = None):

            # print("Input ids shape : ", input_ids.shape)
            output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions

            output_hidden_states = (
                output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
            )

            return_dict = return_dict if return_dict is not None else self.config.use_return_dict

            if input_ids is not None and inputs_embeds is not None:
                raise ValueError("You can't specify both input_ids and inputs_embeds at the same time")
            elif input_ids is not None:
                input_shape = input_ids.size()
                input_ids = input_ids.view(-1, input_shape[-1])

            elif inputs_embeds is not None:
                input_shape = inputs_embeds.size()[:-1]
            else:
                raise ValueError("You have to specify either input_ids or input_embeds")


            if inputs_embeds is None:
                # print("Input ids shape : ", input_ids.shape)
                inputs_embeds = self.embed_tokens(input_ids) * self.embed_scale

            # print("Input shape type : ", type(input_shape))
            # print("Input shape : ", input_shape)
            input_shape = torch.tensor(input_shape)
            # print("Input shape type : ", type(input_shape))
            # print("Input shape : ", input_shape)
            # print("Input shape : ", input_shape.shape)
            embed_pos = self.embed_positions(input_ids)
            # embed_pos = self.embed_positions(input_shape)


            hidden_states = inputs_embeds + embed_pos   
            hidden_states = self.layernorm_embedding(hidden_states)
            hidden_states = F.dropout(hidden_states, p = self.dropout, training=self.training)

            # print("attention mask shape 3 : ", attention_mask.shape)
            if attention_mask is not None:
                attention_mask =  _expand_mask(attention_mask, inputs_embeds.dtype)

            # print("attention mask shape 4 : ", attention_mask.shape)
            encoder_states = () if output_hidden_states else None
            all_attentions = () if output_attentions else None

            if head_mask is not None:
                assert head_mask.size()[0] == (
                    len(self.layers)
                ), f"The head mask should be specified for {len(self.layers)} layers, but it is for {head_mask.size()[0]}."

            for idx, encoder_layer in enumerate(self.layers):
                # print("============Idx : ", idx)

                # if idx in self.fusion_at_layer3:
                #     # print("Acoustic input shape (B) : ", acoustic_input)
                #     # acoustic_input = self.acoustic_transformer(acoustic_input)[-1]   
                #     # print("Acoustic input shape (C) : ", acoustic_input)

                #     # visual_input = self.visual_transformer(visual_input)[-1]
                #     # print("====Idx inside fusion at layer :", idx)
                #     hidden_states = self.MAF_layer3(text_input = hidden_states,
                #                                    acoustic_context = acoustic_input,
                #                                    visual_context = visual_input) 

                # if idx in self.fusion_of_context:
                  
                #   hidden_states = self.context_encoder(hidden_states = hidden_states, context_input_ids = context_input_ids, context_attention_mask = context_attention_mask)
                  

                if idx in self.fusion_at_layer4:
                    # print("Acoustic input shape (B) : ", acoustic_input)
                    # acoustic_input = self.acoustic_transformer(acoustic_input)[-1]   
                    # print("Acoustic input shape (C) : ", acoustic_input)

                    # visual_input = self.visual_transformer(visual_input)[-1]
                    # print("====Idx inside fusion at layer :", idx)
                    
                    hidden_states = self.MAF_layer4(text_input = hidden_states,
                                                   acoustic_context = acoustic_input
                                                   )
                if idx in self.fusion_at_layer5:
                    # print("Acoustic input shape (B) : ", acoustic_input)
                    # acoustic_input = self.acoustic_transformer(acoustic_input)[-1]   
                    # print("Acoustic input shape (C) : ", acoustic_input)

                    # visual_input = self.visual_transformer(visual_input)[-1]
                    # print("====Idx inside fusion at layer :", idx)
                    hidden_states = self.MAF_layer5(text_input = hidden_states,
                                                   visual_context = visual_input)     

                if output_hidden_states:
                    encoder_states = encoder_states + (hidden_states,)

                dropout_probability = random.uniform(0,1)   

                if self.training and (dropout_probability < self.layerdrop):
                    layer_outputs = (None, None)

                else:
                    if self.gradient_checkpointing and self.training:

                        def create_custom_forward(module):
                            def custom_forward(*inputs):
                                return module(*inputs, output_attentions)

                            return custom_forward

                        layer_outputs = torch.utils.checkpoint.checkpoint(
                            create_custom_forward(encoder_layer),
                            hidden_states,
                            attention_mask,
                            (head_mask[idx] if head_mask is not None else None),
                        )        

                    else:
                        # print("Checking Attention mask shape : ", attention_mask.shape)
                        layer_outputs = encoder_layer(
                            hidden_states,
                            attention_mask,
                            layer_head_mask = (head_mask[idx] if head_mask is not None else None),
                            output_attentions = output_attentions
                        )    

                    hidden_states = layer_outputs[0]

                if output_attentions:
                    all_attentions  = all_attentions + (layer_outputs[1],)

            if output_hidden_states:
                encoder_states = encoder_states + (hidden_states,)

            if not return_dict:
                return tuple(v for v in [hidden_states, encoder_states, all_attentions] if v is not None)

            return BaseModelOutput(
                last_hidden_state=hidden_states, hidden_states=encoder_states, attentions=all_attentions
            )    

            # print("Hidden states shape : ", hidden_states)

            # cls = hidden_states.permute(1,0,2)

In [ ]:
class MultimodalBartModel(BartPretrainedModel):
    def __init__(self, config: BartConfig):
        super().__init__(config)

        padding_idx, vocab_size = config.pad_token_id, config.vocab_size
        self.shared = nn.Embedding(vocab_size, config.d_model, padding_idx)

        self.encoder = MultiModalBartEncoder(config, self.shared)
        self.decoder = BartDecoder(config, self.shared)

        self.init_weights()

    def get_input_embeddings(self):
        return self.shared

    def set_input_embeddings(self, value):
        self.shared = value
        self.encoder.embed_tokens = self.shared
        self.decoder.embed_tokens = self.shared

    def get_encoder(self):
        return self.encoder


    def get_decoder(self):
        return self.decoder

    def forward(
        self, 
        input_ids = None,
        attention_mask = None,
        # context_input_ids = None,
        # context_attention_mask = None,
        acoustic_input = None,
        visual_input = None,
        decoder_input_ids = None,
        decoder_attention_mask = None,
        head_mask = None,
        decoder_head_mask = None,
        cross_attn_head_mask = None,
        encoder_outputs = None,
        past_key_values = None,
        inputs_embeds = None,
        decoder_inputs_embeds = None,
        use_cache = None,
        output_attentions = None,
        output_hidden_states = None,
        return_dict = None
    ):

        if decoder_input_ids is None and decoder_inputs_embeds is None:
            decoder_input_ids = shift_tokens_right(
                input_ids, self.config.pad_token_id, self.config.decoder_start_token_id

            )                                

        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )    
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # print("attention mask shape 2 : ", attention_mask.shape)

        if encoder_outputs is None:
            encoder_outputs = self.encoder(
                input_ids = input_ids,
                attention_mask = attention_mask,
                # context_input_ids = context_input_ids,
                # context_attention_mask = context_attention_mask,
                acoustic_input = acoustic_input,
                visual_input = visual_input,
                head_mask = head_mask,
                inputs_embeds = inputs_embeds,
                output_attentions = output_attentions,
                output_hidden_states = output_hidden_states,
                return_dict = return_dict
            )

        elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
            encoder_outputs = BaseModelOutput(
                last_hidden_state=encoder_outputs[0],
                hidden_states=encoder_outputs[1] if len(encoder_outputs) > 1 else None,
                attentions = encoder_outputs[2] if len(encoder_outputs) > 2 else None
            )    

        decoder_outputs = self.decoder(
            input_ids = decoder_input_ids,
            attention_mask = decoder_attention_mask,
            encoder_hidden_states = encoder_outputs[0],
            encoder_attention_mask = attention_mask,
            head_mask = decoder_head_mask,
            cross_attn_head_mask = cross_attn_head_mask,
            past_key_values = past_key_values,
            inputs_embeds = decoder_inputs_embeds,
            use_cache = use_cache,
            output_attentions = output_attentions,
            output_hidden_states = output_hidden_states,
            return_dict = return_dict
        )    

        if not return_dict:
            return decoder_outputs + encoder_outputs

        return Seq2SeqModelOutput(
            last_hidden_state=decoder_outputs.last_hidden_state,
            past_key_values=decoder_outputs.past_key_values,
            decoder_hidden_states=decoder_outputs.hidden_states,
            decoder_attentions=decoder_outputs.attentions,
            cross_attentions=decoder_outputs.cross_attentions,
            encoder_last_hidden_state=encoder_outputs.last_hidden_state,
            encoder_hidden_states=encoder_outputs.hidden_states,
            encoder_attentions=encoder_outputs.attentions

        )

In [ ]:
class MultimodalBartClassification(nn.Module):
    def __init__(
        self, 
        input_dim: int,
        inned_dim: int,
        num_classes: int,
        pooler_dropout: float
    ):
        super().__init__()
        self.dense = nn.Linear(input_dim, inned_dim)
        self.dropout = nn.Dropout(p = pooler_dropout)
        self.out_proj = nn.Linear(inned_dim, num_classes)

    def forward(self, hidden_states):
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.dense(hidden_states)
        hidden_states = torch.tanh(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.out_proj(hidden_states)
        return hidden_states    




class MultimodalBartForSequenceClassification(BartPretrainedModel):
    _keys_to_ignore_on_load_missing = ["encoder.embed_tokens.weight", "decoder.embed_tokens.weight"]

    def __init__(self, config: BartConfig, **kwargs):
        super().__init__(config, **kwargs)
        self.model = MultimodalBartModel(config)
        self.classification_head = MultimodalBartClassification(
            config.d_model,
            config.d_model,
            2,
            config.classifier_dropout
        )
        self.model._init_weights(self.classification_head.dense)
        self.model._init_weights(self.classification_head.out_proj)

    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.tensor] = None,
        # context_input_ids : torch.LongTensor = None,
        # context_attention_mask : Optional[torch.tensor] = None,
        acoustic_input = None,
        visual_input = None,
        decoder_input_ids: Optional[torch.LongTensor] = None,
        decoder_attention_mask: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        decoder_head_mask: Optional[torch.Tensor] = None,
        cross_attn_head_mask: Optional[torch.Tensor] = None,
        encoder_outputs: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None
    ) -> Union[Tuple, Seq2SeqSequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        if labels is not None:
            use_cache = False

        if input_ids is None and inputs_embeds is not None:
            raise NotImplementedError(
                f"Passing input embeddings is currently not supported for {self.__class__.__name__}"
            )

        # print("attention mask shape 1 : ", attention_mask.shape )
        outputs = self.model(
            input_ids,
            attention_mask = attention_mask,
            # context_input_ids = context_input_ids,
            # context_attention_mask = context_attention_mask,
            acoustic_input = acoustic_input,
            visual_input = visual_input,
            decoder_input_ids = decoder_input_ids,
            decoder_attention_mask = decoder_attention_mask,
            head_mask = head_mask,
            decoder_head_mask = decoder_head_mask,
            cross_attn_head_mask = cross_attn_head_mask,
            encoder_outputs = encoder_outputs,
            inputs_embeds = inputs_embeds,
            decoder_inputs_embeds = decoder_inputs_embeds,
            use_cache = use_cache,
            output_attentions = output_attentions,
            output_hidden_states = output_hidden_states,
            return_dict = return_dict

        )    

        hidden_states = outputs[0]

        eos_mask = input_ids.eq(self.config.eos_token_id).to(hidden_states.device)

        if len(torch.unique_consecutive(eos_mask.sum(1))) > 1:
            raise ValueError("All examples must have same number of <eos> tokens")

        sentence_representation = hidden_states[eos_mask, :].view(hidden_states.size(0), -1, hidden_states.size(-1))[
            :, -1, :
        ]    

        logits = self.classification_head(sentence_representation)

        loss = None

        loss_fct = CrossEntropyLoss()
        # print("logits shape : ", logits.shape)
        loss = loss_fct(logits.view(-1, 2), labels.view(-1))
        # print("Loss : ", loss)
        if not return_dict:
            output = (logits,) + outputs[1:]
            return ((loss,) + output) if loss is not None else output


        return Seq2SeqSequenceClassifierOutput(
            loss=loss,
            logits=logits,
            past_key_values=outputs.past_key_values,
            decoder_hidden_states=outputs.decoder_hidden_states,
            decoder_attentions = outputs.decoder_attentions,
            cross_attentions=outputs.cross_attentions,
            encoder_last_hidden_state=outputs.encoder_last_hidden_state,
            encoder_hidden_states=outputs.encoder_hidden_states,
            encoder_attentions=outputs.encoder_attentions
        ) 

In [ ]:
# def audio_video_broadcast(x):
# #     z = torch.empty()
#     temp_all = torch.Tensor()
#     for j in range(x.shape[0]):
#         print("j : ", j)
#         temp_x = x[j,:]
# #         print("Temp x shape : ", temp_x.shape)
#         temp_x = torch.tensor(temp_x, dtype=torch.float)
#         temp_x = torch.broadcast_to(temp_x, (SOURCE_MAX_LEN, temp_x.shape[0]))
# #         print("Temp x shape : ", temp_x.shape)
#         temp_x = temp_x.unsqueeze(0)
# #         print("Temp x shape : ", temp_x.shape)
        
#         if(j==0):
#             temp_all = temp_x
#         else:
#             temp_all = torch.cat([temp_all, temp_x], dim = 0)
        
        
#     return temp_all 

In [ ]:
foldNum

0

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/32/train_dlog_id.p', 'rb') as f:
  train_dlog_id = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/32/test_dlog_id.p', 'rb') as f:
  test_dlog_id = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/32/train_audio_fold_'+str(foldNum)+'.p', 'rb') as f:
  train_audio_data_utterance1 = pickle.load(f)


In [ ]:
type(train_audio_data_utterance1)

list

In [ ]:
len(train_audio_data_utterance1)

552

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/32/test_audio_fold_'+str(foldNum)+'.p', 'rb') as f:
  test_audio_data_utterance1 = pickle.load(f)

In [ ]:
len(test_audio_data_utterance1)

138

In [ ]:
len(test_audio_data_utterance1)

138

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/32/train_video_fold_'+str(foldNum)+'.p', 'rb') as f:
  train_image_data_utterance1 = pickle.load(f)

In [ ]:
len(train_image_data_utterance1)

552

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/32/test_video_fold_'+str(foldNum)+'.p', 'rb') as f:
  test_image_data_utterance1 = pickle.load(f)

In [ ]:
len(test_image_data_utterance1)

138

In [ ]:
tp = torch.ones(4,5)
tp

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [ ]:
torch.zeros(5 - tp.shape[0], 5)

tensor([[0., 0., 0., 0., 0.]])

In [ ]:
torch.cat([tp, torch.zeros(5 - tp.shape[0], 5)])

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0.]])

In [ ]:
def pad_seq(tensor, dim, max_len):
  if max_len > tensor.shape[0] :
    return torch.cat([tensor, torch.zeros(max_len - tensor.shape[0], dim)])
  else:
    return tensor[:max_len]  

In [ ]:
ACOUSTIC_DIM

768

In [ ]:
train_audio_data_utterance1 = torch.stack([pad_seq(torch.tensor(a, dtype = torch.float),
                                                   dim = ACOUSTIC_DIM,
                                                   max_len = ACOUSTIC_MAX_LEN)
                                                  for a in train_audio_data_utterance1], 0)
train_audio_data_utterance1.shape

torch.Size([552, 1000, 768])

In [ ]:
test_audio_data_utterance1 = torch.stack([pad_seq(torch.tensor(a, dtype = torch.float),
                                                   dim = ACOUSTIC_DIM,
                                                   max_len = ACOUSTIC_MAX_LEN)
                                                  for a in test_audio_data_utterance1], 0)
test_audio_data_utterance1.shape

torch.Size([138, 1000, 768])

In [ ]:
VISUAL_DIM

2048

In [ ]:
VISUAL_MAX_LEN

480

In [ ]:
train_image_data_utterance1 = torch.stack([pad_seq(torch.tensor(a, dtype = torch.float),
                                                   dim = VISUAL_DIM,
                                                   max_len = VISUAL_MAX_LEN)
                                                  for a in train_image_data_utterance1], 0)
train_image_data_utterance1.shape

torch.Size([552, 480, 2048])

In [ ]:
test_image_data_utterance1 = torch.stack([pad_seq(torch.tensor(a, dtype = torch.float),
                                                   dim = VISUAL_DIM,
                                                   max_len = VISUAL_MAX_LEN)
                                                  for a in test_image_data_utterance1], 0)
test_image_data_utterance1.shape

torch.Size([138, 480, 2048])

In [ ]:
# path = "/content/drive/MyDrive/Colab Notebooks/32/datasetTrue_original/sarcasmDataset_speaker_dependent_True.npz"
# data2 = np.load(path, mmap_mode=True)   

# train_audio_data_utterance1 = data2['feautesUA_train'][foldNum]
# train_image_data_utterance1 = data2['feautesUV_train'][foldNum]

# test_audio_data_utterance1 = data2['feautesUA_test'][foldNum]
# test_image_data_utterance1 = data2['feautesUV_test'][foldNum]

In [ ]:
model = MultimodalBartForSequenceClassification.from_pretrained("facebook/bart-base")
print(model)

tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')
print("Tokenizer : ", tokenizer)

num_param = sum(p.numel() for p in model.parameters())
print("Total parameters : ", num_param/1e6)

Some weights of MultimodalBartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.out_proj.bias', 'encoder.MAF_layer4.acoustic_context_attention.w1_v.weight', 'encoder.MAF_layer5.visual_context_attention.attention_layer.out_proj.weight', 'encoder.MAF_layer4.acoustic_context_attention.attention_layer.in_proj_bias', 'encoder.MAF_layer5.visual_context_attention.w2_k.weight', 'encoder.MAF_layer5.visual_context_attention.attention_layer.in_proj_bias', 'encoder.MAF_layer5.visual_context_attention.attention_layer.out_proj.bias', 'encoder.MAF_layer5.visual_context_attention.u_k.weight', 'encoder.MAF_layer5.visual_gate.bias', 'encoder.MAF_layer4.acoustic_context_attention.w2_k.weight', 'encoder.MAF_layer4.acoustic_context_attention.u_k.weight', 'encoder.MAF_layer4.acoustic_context_attention.w1_k.weight', 'encoder.MAF_layer5.visual_gate.weight', 'encoder.MAF_layer5.visual_context_attention.u_v.weight', 'e

MultimodalBartForSequenceClassification(
  (model): MultimodalBartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): MultiModalBartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm

Tokenizer :  BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})
Total parameters :  152.172706


In [ ]:
cnt = 0
for name, param in model.named_parameters():
    print("Count : ", cnt, " name : ", name)
    cnt+=1

Count :  0  name :  model.shared.weight
Count :  1  name :  model.encoder.embed_positions.weight
Count :  2  name :  model.encoder.layers.0.self_attn.k_proj.weight
Count :  3  name :  model.encoder.layers.0.self_attn.k_proj.bias
Count :  4  name :  model.encoder.layers.0.self_attn.v_proj.weight
Count :  5  name :  model.encoder.layers.0.self_attn.v_proj.bias
Count :  6  name :  model.encoder.layers.0.self_attn.q_proj.weight
Count :  7  name :  model.encoder.layers.0.self_attn.q_proj.bias
Count :  8  name :  model.encoder.layers.0.self_attn.out_proj.weight
Count :  9  name :  model.encoder.layers.0.self_attn.out_proj.bias
Count :  10  name :  model.encoder.layers.0.self_attn_layer_norm.weight
Count :  11  name :  model.encoder.layers.0.self_attn_layer_norm.bias
Count :  12  name :  model.encoder.layers.0.fc1.weight
Count :  13  name :  model.encoder.layers.0.fc1.bias
Count :  14  name :  model.encoder.layers.0.fc2.weight
Count :  15  name :  model.encoder.layers.0.fc2.bias
Count :  16  

In [ ]:
cnt = 0
for name, param in model.named_parameters():
    # print("Count : ", cnt, " name : ", name)
    # if(cnt>=100 and cnt<=177):
    # if(cnt>=66 and cnt<=151):
    # if(cnt>=100 and cnt<=179):
    # if(cnt>=100 and cnt<=155):
    # if(cnt>=100 and cnt<=189):
    if(cnt>=100 and cnt<=129):
    # if(cnt>=100 and cnt<=125):
        param.requires_grad = True
        print("Count : ", cnt, " name : ", name)
    # elif(cnt>=337):
    # elif(cnt>=343):
    # elif(cnt>=315):
    # elif(cnt>=329 and cnt<=332):
    #     param.requires_grad = True
    #     print("Count : ", cnt, " name : ", name)
    elif(cnt>=289):
    # elif(cnt>=311):
    # elif(cnt>=285):  
        param.requires_grad = True
        print("Count : ", cnt, " name : ", name)

    else:
        param.requires_grad = False    
    cnt+=1
        

num_param = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total trainanable parameters : ", num_param/1e6)

Count :  100  name :  model.encoder.MAF_layer4.acoustic_context_transform.weight
Count :  101  name :  model.encoder.MAF_layer4.acoustic_context_attention.attention_layer.in_proj_weight
Count :  102  name :  model.encoder.MAF_layer4.acoustic_context_attention.attention_layer.in_proj_bias
Count :  103  name :  model.encoder.MAF_layer4.acoustic_context_attention.attention_layer.out_proj.weight
Count :  104  name :  model.encoder.MAF_layer4.acoustic_context_attention.attention_layer.out_proj.bias
Count :  105  name :  model.encoder.MAF_layer4.acoustic_context_attention.u_k.weight
Count :  106  name :  model.encoder.MAF_layer4.acoustic_context_attention.w1_k.weight
Count :  107  name :  model.encoder.MAF_layer4.acoustic_context_attention.w2_k.weight
Count :  108  name :  model.encoder.MAF_layer4.acoustic_context_attention.u_v.weight
Count :  109  name :  model.encoder.MAF_layer4.acoustic_context_attention.w1_v.weight
Count :  110  name :  model.encoder.MAF_layer4.acoustic_context_attention

In [ ]:
foldNum

0

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/32/json_file_fold.p", "rb") as f:
    text_file = pickle.load(f)

train_text = text_file["json_file_list_" + str(foldNum) +"_train"]
test_text =  text_file["json_file_list_" + str(foldNum) +"_test"]

In [ ]:
# Trainlen = 483

In [ ]:
print(type(train_audio_data_utterance1))
print(train_audio_data_utterance1.shape)

<class 'torch.Tensor'>
torch.Size([552, 1000, 768])


In [ ]:
# combined_train_data = []
# for j in range(len(train_text)):
#   temp_text = train_text[j]
#   temp_audio = train_audio_data_utterance1[j]
#   temp_image = train_image_data_utterance1[j]

#   temp_list = []
#   temp_list.append(temp_text)
#   temp_list.append(temp_audio)
#   temp_list.append(temp_image)

#   combined_train_data.append(temp_list)

# random.shuffle(combined_train_data)

In [ ]:
# train_text2 = []
# train_audio_utterance2 = []
# train_image_utterance2 = []

# for j in combined_train_data:
#   train_text2.append(j[0])
#   train_audio_utterance2.append(j[1])
#   train_image_utterance2.append(j[2])





In [ ]:
# print(len(train_text2))
# print(len(train_audio_utterance2))
# print(len(train_image_utterance2))

In [ ]:
# train_audio_data_utterance2 = torch.tensor(train_audio_utterance2)
# train_audio_data_utterance2.shape

In [ ]:
# train_image_data_utterance2 = torch.tensor(train_image_utterance2)
# train_image_data_utterance2.shape

In [ ]:
# train_audio_data_utterance = torch.tensor(train_audio_data_utterance2)[:Trainlen]
# # train_audio_data_utterance = train_audio_data_utterance.unsqueeze(dim = 1)
# train_audio_data_utterance.shape

In [ ]:
train_audio_data_utterance = torch.tensor(train_audio_data_utterance1)
# train_audio_data_utterance = train_audio_data_utterance.unsqueeze(dim = 1)
train_audio_data_utterance.shape

torch.Size([552, 1000, 768])

In [ ]:
# train_audio_data_utterance = torch.tensor(train_audio_data_utterance1)[:Trainlen]
# # train_audio_data_utterance = train_audio_data_utterance.unsqueeze(dim = 1)
# train_audio_data_utterance.shape

In [ ]:
# train_image_data_utterance = torch.tensor(train_image_data_utterance2)[:Trainlen]
# # train_image_data_utterance = train_image_data_utterance.unsqueeze(dim = 1)
# train_image_data_utterance.shape

In [ ]:
train_image_data_utterance = torch.tensor(train_image_data_utterance1)
# train_image_data_utterance = train_image_data_utterance.unsqueeze(dim = 1)
train_image_data_utterance.shape

torch.Size([552, 480, 2048])

In [ ]:
# train_image_data_utterance = torch.tensor(train_image_data_utterance1)[:Trainlen]
# # train_image_data_utterance = train_image_data_utterance.unsqueeze(dim = 1)
# train_image_data_utterance.shape

In [ ]:
# valid_audio_data_utterance = torch.tensor(train_audio_data_utterance2)[Trainlen:]
# valid_audio_data_utterance.shape

In [ ]:
len(test_text)

138

In [ ]:
test_image_data_utterance1.shape

torch.Size([138, 480, 2048])

In [ ]:
test_audio_data_utterance1.shape

torch.Size([138, 1000, 768])

In [ ]:
# combined_test_data = []

# for j in range(len(test_text)):
#   temp_text = test_text[j]
#   temp_audio = test_audio_data_utterance1[j]
#   temp_image = test_image_data_utterance1[j]

#   temp_list = []
#   temp_list.append(temp_text)
#   temp_list.append(temp_audio)
#   temp_list.append(temp_image)

#   combined_test_data.append(temp_list)

# random.shuffle(combined_test_data)


In [ ]:
# test_text2 = []
# test_audio_utterance2 = []
# test_image_utterance2 = []

# for j in combined_test_data:
#   test_text2.append(j[0])
#   test_audio_utterance2.append(j[1])
#   test_image_utterance2.append(j[2])


In [ ]:
# VALIDLEN = 69

In [ ]:
# len(test_audio_utterance2)

In [ ]:
# test_audio_utterance2[0].shape

In [ ]:
# torch.stack(test_audio_utterance2).shape

In [ ]:
# valid_audio_data_utterance = test_audio_data_utterance1[:VALIDLEN]
# valid_audio_data_utterance.shape

In [ ]:
# test_audio_data_utterance = test_audio_data_utterance1[VALIDLEN:]
# test_audio_data_utterance.shape

In [ ]:
# valid_image_data_utterance = torch.tensor(train_image_data_utterance2)[Trainlen:]
# valid_image_data_utterance.shape

In [ ]:
# valid_image_data_utterance = test_image_data_utterance1[:VALIDLEN]
# valid_image_data_utterance.shape

In [ ]:
# test_image_data_utterance = test_image_data_utterance1[VALIDLEN:]
# test_image_data_utterance.shape

In [ ]:
# test_audio_data_utterance = torch.tensor(test_audio_data_utterance1)
# # test_audio_data_utterance = test_audio_data_utterance.unsqueeze(dim = 1)
# test_audio_data_utterance.shape

In [ ]:
# test_image_data_utterance = torch.tensor(test_image_data_utterance1)
# # test_image_data_utterance = test_image_data_utterance.unsqueeze(dim = 1)
# test_image_data_utterance.shape

In [ ]:
   

# # print(len(text_file[train_text]))
# # print(train_text)
# # print(len(text_file[test_text]))

# train_audio_broadcast_utterance = audio_video_broadcast(train_audio_data_utterance)

# print("train_audio_broadcast_utterance complete : ", train_audio_broadcast_utterance.shape)
# train_image_broadcast_utterance = audio_video_broadcast(train_image_data_utterance)
# print("train_image_broadcast_utterance complete : ",train_image_broadcast_utterance.shape)

# valid_audio_broadcast_utterance = audio_video_broadcast(valid_audio_data_utterance)
# print("valid_audio_broadcast_utterance complete : ", valid_audio_broadcast_utterance.shape)

# valid_image_broadcast_utterance = audio_video_broadcast(valid_image_data_utterance)
# print('valid_image_broadcast_utterance complete : ', valid_image_broadcast_utterance.shape)

# test_audio_broadcast_utterance = audio_video_broadcast(test_audio_data_utterance)
# print("test_audio_broadcast_utterance complete : ",test_audio_broadcast_utterance.shape)
# test_image_broadcast_utterance = audio_video_broadcast(test_image_data_utterance)
# print("test_image_broadcast_utterance complete : ",test_image_broadcast_utterance.shape)

In [ ]:
tokenizer

BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [ ]:
p = {
        'additional_special_tokens' : ['[CONTEXT]', '[UTTERANCE]']
    }

tokenizer.add_special_tokens(p)

tokenizer

BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False), 'additional_special_tokens': ['[CONTEXT]', '[UTTERANCE]']})

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50267, 768)

In [ ]:
# def prepare_dataset_context(text_data):
                    

#             context = []
#             # labels = []
#             for i in range(len(text_data)):
#                 data_point = text_data[i]

#                 # example_speaker = data_point['speaker']
#                 # example_utterance = data_point['utterance']
#                 # temp_label = int(data_point['sarcasm'])

#                 # example_context = '[CONTEXT] '
#                 example_context = ''

#                 temp_len = len(data_point['context_speakers'])
#                 cnt = 0
#                 print("Temp len : ", temp_len)
#                 for speaker, utterance in list(zip(data_point['context_speakers'], data_point['context'])):
#                     print("count : ", cnt)
#                     if(cnt == temp_len - 1):
#                       example_context = example_context + speaker.upper() + " : " + utterance
#                     else:
#                       example_context = example_context + speaker.upper() + " : " + utterance + " , "
#                     cnt+=1

                    

                
#                 # print(example_dialog)
#                 example_context = re.sub(' +', ' ', example_context)

#                 context.append(example_context)
#                 # labels.append(temp_label)

#             # df = pd.DataFrame(dialog, columns=['dialog'])

#             # labels = torch.tensor(labels, dtype=torch.long)

            

#             enc = bert_tokenizer(context, max_length = SOURCE_MAX_LEN, padding = 'max_length', truncation = True)

#             # df['audio_features'] = acoustic_data
#             # df['visual_features'] = visual_data

#             return torch.tensor(enc['input_ids'], dtype=torch.long), torch.tensor(enc['attention_mask'], dtype=torch.bool)


In [ ]:
def prepare_dataset(text_data):
                    

            dialog = []
            labels = []
            sample_utterances = []
            for i in range(len(text_data)):
                data_point = text_data[i]

                example_speaker = data_point['speaker']
                example_utterance = data_point['utterance']
                temp_label = int(data_point['sarcasm'])
                sample_utterances.append(data_point)

                # example_dialog = '[CONTEXT] '
                # example_dialog = '[TARGET] '
                example_dialog = '[CONTEXT] '


                for speaker, utterance in list(zip(data_point['context_speakers'], data_point['context'])):
                    example_dialog = example_dialog + speaker.upper() + " : " + utterance + " | "

                example_dialog = example_dialog + ' [UTTERANCE] ' + example_speaker + " : " + example_utterance + " | "
                # example_dialog = example_dialog + example_speaker + " : " + example_utterance
                # example_dialog = example_dialog + example_speaker + " : " + example_utterance 
                # print(example_dialog)
                example_dialog = re.sub(' +', ' ', example_dialog)

                dialog.append(example_dialog)
                labels.append(temp_label)

                print(data_point)
                print(temp_label)

            # df = pd.DataFrame(dialog, columns=['dialog'])

            labels = torch.tensor(labels, dtype=torch.long)

            

            enc = tokenizer(dialog, max_length = SOURCE_MAX_LEN, padding = 'max_length', truncation = True)

            # df['audio_features'] = acoustic_data
            # df['visual_features'] = visual_data

            return torch.tensor(enc['input_ids'], dtype=torch.long), torch.tensor(enc['attention_mask'], dtype=torch.bool), sample_utterances, labels


In [ ]:
tokenizer

BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False), 'additional_special_tokens': ['[CONTEXT]', '[UTTERANCE]']})

In [ ]:
len(train_text)

552

In [ ]:
train_text

[{'utterance': "Since it's not bee season, you can have my epinephrine.",
  'speaker': 'SHELDON',
  'context': ['Here we go. Pad thai, no peanuts.',
   'But does it have peanut oil?',
   "I'm not sure. Everyone keep an eye on Howard in case he starts to swell up."],
  'context_speakers': ['LEONARD', 'HOWARD', 'LEONARD'],
  'show': 'BBT',
  'sarcasm': False},
 {'utterance': 'Lois Lane is falling, accelerating at an initial rate of 32 feet per second per second. Superman swoops down to save her by reaching out two arms of steel. Miss Lane, who is now traveling at approximately 120 miles an hour, hits them and is immediately sliced into three equal pieces.',
  'speaker': 'SHELDON',
  'context': ['A marathon? How many Superman movies are there?',
   "You're kidding, right?",
   'I do like the one where Lois Lane falls from the helicopter and Superman swooshes down and catches her. Which one was that?',
   'One.',
   'You realize that scene was rife with scientific inaccuracy.',
   "Yes, I 

In [ ]:
train_text_input_ids1, train_text_attention_mask1, train_utterances, train_ground_truth1 = prepare_dataset(train_text)
train_ground_truth1.shape

{'utterance': "Since it's not bee season, you can have my epinephrine.", 'speaker': 'SHELDON', 'context': ['Here we go. Pad thai, no peanuts.', 'But does it have peanut oil?', "I'm not sure. Everyone keep an eye on Howard in case he starts to swell up."], 'context_speakers': ['LEONARD', 'HOWARD', 'LEONARD'], 'show': 'BBT', 'sarcasm': False}
0
{'utterance': 'Lois Lane is falling, accelerating at an initial rate of 32 feet per second per second. Superman swoops down to save her by reaching out two arms of steel. Miss Lane, who is now traveling at approximately 120 miles an hour, hits them and is immediately sliced into three equal pieces.', 'speaker': 'SHELDON', 'context': ['A marathon? How many Superman movies are there?', "You're kidding, right?", 'I do like the one where Lois Lane falls from the helicopter and Superman swooshes down and catches her. Which one was that?', 'One.', 'You realize that scene was rife with scientific inaccuracy.', "Yes, I know, men can't fly.", "Let's assume

torch.Size([552])

In [ ]:
# train_text_input_ids1, train_text_attention_mask1, train_ground_truth1 = prepare_dataset_utterance(train_text)
# train_ground_truth1.shape

In [ ]:
train_text_input_ids = train_text_input_ids1
train_text_input_ids.shape

torch.Size([552, 500])

In [ ]:
# train_text_input_ids = train_text_input_ids1[:Trainlen]
# train_text_input_ids.shape

In [ ]:
train_text_attention_mask = train_text_attention_mask1
train_text_attention_mask.shape

torch.Size([552, 500])

In [ ]:
# train_text_attention_mask = train_text_attention_mask1[:Trainlen]
# train_text_attention_mask.shape

In [ ]:
train_ground_truth = train_ground_truth1
train_ground_truth.shape

torch.Size([552])

In [ ]:
train_utterances

[{'utterance': "Since it's not bee season, you can have my epinephrine.",
  'speaker': 'SHELDON',
  'context': ['Here we go. Pad thai, no peanuts.',
   'But does it have peanut oil?',
   "I'm not sure. Everyone keep an eye on Howard in case he starts to swell up."],
  'context_speakers': ['LEONARD', 'HOWARD', 'LEONARD'],
  'show': 'BBT',
  'sarcasm': False},
 {'utterance': 'Lois Lane is falling, accelerating at an initial rate of 32 feet per second per second. Superman swoops down to save her by reaching out two arms of steel. Miss Lane, who is now traveling at approximately 120 miles an hour, hits them and is immediately sliced into three equal pieces.',
  'speaker': 'SHELDON',
  'context': ['A marathon? How many Superman movies are there?',
   "You're kidding, right?",
   'I do like the one where Lois Lane falls from the helicopter and Superman swooshes down and catches her. Which one was that?',
   'One.',
   'You realize that scene was rife with scientific inaccuracy.',
   "Yes, I 

In [ ]:
# train_ground_truth = train_ground_truth1[:Trainlen]
# train_ground_truth.shape

In [ ]:
# train_ground_truth = train_ground_truth1[:Trainlen]
# train_ground_truth.shape

In [ ]:
# train_ground_truth

In [ ]:
# context_input_ids, context_attention_mask = prepare_dataset_context(train_text)
# print(context_input_ids.shape)

In [ ]:
# context_attention_mask.shape

In [ ]:
print("TYPE : train_text_input_ids : ", type(train_text_input_ids))

TYPE : train_text_input_ids :  <class 'torch.Tensor'>


In [ ]:

# train_context_input_ids = context_input_ids
# train_context_input_ids.shape

In [ ]:

# train_context_input_ids = context_input_ids[:Trainlen]
# train_context_input_ids.shape

In [ ]:
# train_context_attention_mask = context_attention_mask
# train_context_attention_mask.shape

In [ ]:
# train_context_attention_mask = context_attention_mask[:Trainlen]
# train_context_attention_mask.shape

In [ ]:
# valid_text_input_ids = train_text_input_ids1[Trainlen:]
# valid_text_input_ids.shape

In [ ]:

# valid_text_attention_mask = train_text_attention_mask1[Trainlen:]
# valid_text_attention_mask.shape

In [ ]:
# valid_ground_truth = train_ground_truth1[Trainlen:]
# valid_ground_truth.shape

In [ ]:
# valid_text_input_ids, valid_text_attention_mask, valid_ground_truth = prepare_dataset_utterance(valid_text)
# valid_ground_truth.shape

In [ ]:
# valid_context_input_ids = context_input_ids[Trainlen:]
# valid_context_input_ids.shape

In [ ]:
# valid_context_attention_mask = context_attention_mask[Trainlen:]
# valid_context_attention_mask.shape

In [ ]:
# test_text_input_ids, test_text_attention_mask, test_ground_truth = prepare_dataset_utterance(test_text)
# test_ground_truth.shape

In [ ]:
test_text_input_ids, test_text_attention_mask, test_utterances1, test_ground_truth = prepare_dataset(test_text)
test_ground_truth.shape

{'utterance': "It's just a privilege to watch your mind at work.", 'speaker': 'SHELDON', 'context': ['I never would have identified the fingerprints of string theory in the aftermath of the Big Bang.', "My apologies. What's your plan?"], 'context_speakers': ['LEONARD', 'SHELDON'], 'show': 'BBT', 'sarcasm': True}
1
{'utterance': "I don't think I'll be able to stop thinking about it.", 'speaker': 'PENNY', 'context': ['This is one of my favorite places to kick back after a quest. They have a great house ale.', 'Cool tiger.', "Yeah, I've had him since level ten.", 'His name is Buttons.', 'Anyway, if you had your own game character we could hang out, maybe go on a quest.', 'That sounds interesting.', "You'll think about it?"], 'context_speakers': ['HOWARD', 'PENNY', 'HOWARD', 'HOWARD', 'HOWARD', 'PENNY', 'HOWARD'], 'show': 'BBT', 'sarcasm': True}
1
{'utterance': "I'm just inferring this is a couch because the evidence suggests the coffee table is having a tiny garage sale.", 'speaker': 'SHE

torch.Size([138])

In [ ]:
# valid_id = test_text_input_ids[:VALID_LEN]
# valid_id.shape

In [ ]:
# test_id = test_text_input_ids[VALIDLEN:]
# test_id.shape

In [ ]:
# valid_mask = test_text_attention_mask[:VALIDLEN]
# valid_mask.shape

In [ ]:
# test_mask = test_text_attention_mask[VALIDLEN:]
# test_mask.shape

In [ ]:
# valid_truth = test_ground_truth[:VALIDLEN]
# valid_truth.shape

In [ ]:
# test_truth = test_ground_truth[VALIDLEN:]
# test_truth.shape

In [ ]:
# test_context_input_ids, test_context_attention_mask = prepare_dataset_context(test_text)
# test_context_input_ids.shape

In [ ]:
# valid_context_id = test_context_input_ids[:VALIDLEN]
# valid_context_id.shape

In [ ]:
# test_context_id = test_context_input_ids[VALIDLEN:]
# test_context_id.shape

In [ ]:
# test_context_attention_mask.shape

In [ ]:
# valid_context_mask = test_context_attention_mask[:VALIDLEN]
# valid_context_mask.shape

In [ ]:
# test_context_mask = test_context_attention_mask[VALIDLEN:]
# test_context_mask.shape

In [ ]:
# tokenizer.add_tokens(['[CONTEXT]', '[TARGET]'], special_tokens = True)
# model.resize_token_embeddings(len(tokenizer))

In [ ]:
test_audio_data_utterance1.shape

torch.Size([138, 1000, 768])

In [ ]:
test_image_data_utterance1.shape

torch.Size([138, 480, 2048])

In [ ]:
print(test_text_input_ids.shape)
print(test_text_attention_mask.shape)
print(test_ground_truth.shape)

torch.Size([138, 500])
torch.Size([138, 500])
torch.Size([138])


In [ ]:
# print(test_context_input_ids.shape)
# print(test_context_attention_mask.shape)


In [ ]:
# test_input_data = []


# for j in range(test_ground_truth.shape[0]):
#   temp_list = []
#   temp_list.append(test_text_input_ids[j])
#   temp_list.append(test_text_attention_mask[j])
#   temp_list.append(test_context_input_ids[j])
#   temp_list.append(test_context_attention_mask[j])
#   temp_list.append(test_audio_data_utterance1[j])
#   temp_list.append(test_image_data_utterance1[j])

#   test_input_data.append(temp_list)

  


In [ ]:
# print(type(test_input_data))
# print(len(test_input_data))

In [ ]:
# test_output_data = test_ground_truth.tolist()
# print(type(test_output_data))
# print(len(test_output_data))

In [ ]:
# X_valid, X_test, Y_valid, Y_test = train_test_split(
#     test_input_data, test_output_data, test_size = 0.5, stratify = test_output_data
# )

In [ ]:
# len(X_valid)

In [ ]:
# len(X_test)

In [ ]:
# len(Y_valid)

In [ ]:
# len(Y_test)

In [ ]:
# valid_text_input_ids = []
# valid_text_attention_mask = []
# valid_context_input_ids = []
# valid_context_attention_mask = []

# valid_audio_data = []
# valid_image_data = []

# for j in range(len(X_valid)):
#   valid_text_input_ids.append(X_valid[j][0])
#   valid_text_attention_mask.append(X_valid[j][1])
  
#   valid_context_input_ids.append(X_valid[j][2])
#   valid_context_attention_mask.append(X_valid[j][3])
  
#   valid_audio_data.append(X_valid[j][4])

#   valid_image_data.append(X_valid[j][5])

# print(len(valid_text_input_ids))
# print(len(valid_text_attention_mask))
# print(len(valid_context_input_ids))
# print(len(valid_context_attention_mask))
# print(len(valid_audio_data))
# print(len(valid_image_data))

In [ ]:
# valid_text_input_ids[0].shape

In [ ]:
# valid_text_input_ids = torch.stack(valid_text_input_ids)
# print(valid_text_input_ids.shape)
# valid_text_attention_mask = torch.stack(valid_text_attention_mask)
# print(valid_text_attention_mask.shape)
# valid_context_input_ids = torch.stack(valid_context_input_ids)
# print(valid_context_input_ids.shape)
# valid_context_attention_mask = torch.stack(valid_context_attention_mask)
# print(valid_context_attention_mask.shape)
# valid_audio_data = torch.stack(valid_audio_data)
# print(valid_audio_data.shape)
# valid_image_data = torch.stack(valid_image_data)
# print(valid_image_data.shape)

In [ ]:
# valid_ground_truth = torch.tensor(Y_valid)
# valid_ground_truth.shape

In [ ]:
# test_text_id = []
# test_text_mask = []

# test_context_id = []
# test_context_mask = []

# test_audio_data = []
# test_image_data = []

# for j in range(len(X_test)):
#   test_text_id.append(X_test[j][0])
#   test_text_mask.append(X_test[j][1])

#   test_context_id.append(X_test[j][2])
#   test_context_mask.append(X_test[j][3])

#   test_audio_data.append(X_test[j][4])

#   test_image_data.append(X_test[j][5])

# test_text_id = torch.stack(test_text_id)
# print(test_text_id.shape)
# test_text_mask = torch.stack(test_text_mask)
# print(test_text_mask.shape)
# test_context_id = torch.stack(test_context_id)
# print(test_context_id.shape)
# test_context_mask = torch.stack(test_context_mask)
# print(test_context_mask.shape)
# test_audio_data = torch.stack(test_audio_data)
# print(test_audio_data.shape)
# test_image_data = torch.stack(test_image_data)
# print(test_image_data.shape)  

In [ ]:
# test_ground_truth = torch.tensor(Y_test)
# print(test_ground_truth.shape)

In [ ]:

# tokenizer

In [ ]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<pad>', '<mask>', '[CONTEXT]', '[UTTERANCE]']

In [ ]:
tokenizer.all_special_tokens


['<s>', '</s>', '<unk>', '<pad>', '<mask>', '[CONTEXT]', '[UTTERANCE]']

In [ ]:
print(tokenizer.all_special_ids)

[0, 2, 3, 1, 50264, 50265, 50266]


In [ ]:
# train_audio_broadcast_utterance.shape

In [ ]:
# valid_audio_data_utterance = test_audio_data_utterance[:VALID_LEN, :, :]
# valid_audio_data_utterance.shape

In [ ]:
# valid_image_data_utterance = test_image_data_utterance[:VALID_LEN, :, :]
# valid_image_data_utterance.shape

In [ ]:




# test_audio_data_utterance = test_audio_data_utterance[VALID_LEN:, :, :]
# test_audio_data_utterance.shape

In [ ]:


# test_image_data_utterance = test_image_data_utterance[VALID_LEN:, :, :]
# test_image_data_utterance.shape

In [ ]:
len(test_utterances1)

138

In [ ]:
test_ground_truth.shape[0]

138

In [ ]:
test_input_data = []


for j in range(test_ground_truth.shape[0]):
  temp_list = []
  temp_list.append(test_text_input_ids[j])
  temp_list.append(test_text_attention_mask[j])

  temp_list.append(test_audio_data_utterance1[j])
  temp_list.append(test_image_data_utterance1[j])
  temp_list.append(test_utterances1[j])
  temp_list.append(test_dlog_id[j])

  test_input_data.append(temp_list)

  


In [ ]:
print(type(test_input_data))
print(len(test_input_data))

<class 'list'>
138


In [ ]:
test_output_data = test_ground_truth.tolist()
print(type(test_output_data))
print(len(test_output_data))

<class 'list'>
138


In [ ]:
X_valid, X_test, Y_valid, Y_test = train_test_split(
    test_input_data, test_output_data, test_size = 0.5, stratify = test_output_data
)

In [ ]:
len(X_valid)

69

In [ ]:
len(X_test)

69

In [ ]:
len(Y_valid)

69

In [ ]:
len(Y_test)

69

In [ ]:
valid_text_input_ids = []
valid_text_attention_mask = []
valid_context_input_ids = []
valid_context_attention_mask = []

valid_audio_data = []
valid_image_data = []



valid_utterances_data = []
valid_did = []

for j in range(len(X_valid)):
  valid_text_input_ids.append(X_valid[j][0])
  valid_text_attention_mask.append(X_valid[j][1])
  
  
  valid_audio_data.append(X_valid[j][2])

  valid_image_data.append(X_valid[j][3])
  valid_utterances_data.append(X_valid[j][4])
  valid_did.append(X_valid[j][5])

print(len(valid_text_input_ids))
print(len(valid_text_attention_mask))

print(len(valid_audio_data))
print(len(valid_image_data))

print(len(valid_utterances_data))
print(len(valid_did))

69
69
69
69
69
69


In [ ]:
valid_text_input_ids[0].shape

torch.Size([500])

In [ ]:
valid_text_input_ids = torch.stack(valid_text_input_ids)
print(valid_text_input_ids.shape)
valid_text_attention_mask = torch.stack(valid_text_attention_mask)
print(valid_text_attention_mask.shape)

valid_audio_data = torch.stack(valid_audio_data)
print(valid_audio_data.shape)
valid_image_data = torch.stack(valid_image_data)
print(valid_image_data.shape)

torch.Size([69, 500])
torch.Size([69, 500])
torch.Size([69, 1000, 768])
torch.Size([69, 480, 2048])


In [ ]:
valid_ground_truth = torch.tensor(Y_valid)
valid_ground_truth.shape

torch.Size([69])

In [ ]:
test_text_id = []
test_text_mask = []

test_context_id = []
test_context_mask = []

test_audio_data = []
test_image_data = []

test_utterances_data = []
test_did = []
for j in range(len(X_test)):
  test_text_id.append(X_test[j][0])
  test_text_mask.append(X_test[j][1])


  test_audio_data.append(X_test[j][2])

  test_image_data.append(X_test[j][3])
  test_utterances_data.append(X_test[j][4])
  test_did.append(X_test[j][5])

test_text_id = torch.stack(test_text_id)
print(test_text_id.shape)
test_text_mask = torch.stack(test_text_mask)
print(test_text_mask.shape)

test_audio_data = torch.stack(test_audio_data)
print(test_audio_data.shape)
test_image_data = torch.stack(test_image_data)
print(test_image_data.shape)  

print(len(test_utterances_data))
print(len(test_did))

torch.Size([69, 500])
torch.Size([69, 500])
torch.Size([69, 1000, 768])
torch.Size([69, 480, 2048])
69
69


In [ ]:
test_ground_truth = torch.tensor(Y_test)
print(test_ground_truth.shape)

torch.Size([69])


In [ ]:
valid_utterances_data[0]

{'utterance': 'No, she is going to sit here where itÕs a 112 degrees and eat enchiladas.',
 'speaker': 'DOROTHY',
 'context': ['Its Miami in June, only cats are wearing fur!',
  'You going out?'],
 'context_speakers': ['DOROTHY', 'ROSE'],
 'show': 'GOLDENGIRLS',
 'sarcasm': True}

In [ ]:
test_did[0]

'1_10748'

In [ ]:
test_utterances_data[0]

{'utterance': "There they go, fighting again. You'd never hear her talk that way to Sauvignon Blanc.",
 'speaker': 'AMY',
 'context': ['Howard and Bernadette, Raj and his girlfriend, Penny and Chardonnay, Penny and you.',
  "There isn't any ranking.",
  "And if there were, we wouldn't be at the bottom of it.",
  'Yeah, and actually, I drink Sauvignon Blanc.',
  "That's the part you have a problem with?",
  'Relax.'],
 'context_speakers': ['SHELDON',
  'LEONARD',
  'LEONARD',
  'PENNY',
  'LEONARD',
  'PENNY'],
 'show': 'BBT',
 'sarcasm': True}

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/32/test_did.p', 'wb') as f:
  pickle.dump(test_did, f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/32/valid_did.p', 'wb') as f:
  pickle.dump(valid_did, f)

In [ ]:
class MultimodalSarcasmDataset(Dataset):
    # def __init__(self, utterance_input_ids, utterance_attention_mask, context_input_ids, context_attention_mask, acoustic_data, visual_data, labels):
    def __init__(self, utterance_input_ids, utterance_attention_mask, acoustic_data, visual_data, labels):

        self.utterance_input_ids = utterance_input_ids
        self.utterance_attention_mask = utterance_attention_mask
        # self.context_input_ids = context_input_ids
        # self.context_attention_mask = context_attention_mask
        # self.context_attention_mask
        self.acoustic_data = acoustic_data
        self.visual_data = visual_data
        self.labels = labels

    def __len__(self):
        return len(self.utterance_input_ids)

    def __getitem__(self, idx):
        # return self.utterance_input_ids[idx], self.utterance_attention_mask[idx], self.context_input_ids[idx], self.context_attention_mask[idx], self.acoustic_data[idx], self.visual_data[idx], self.labels[idx]
        return self.utterance_input_ids[idx], self.utterance_attention_mask[idx],  self.acoustic_data[idx], self.visual_data[idx], self.labels[idx]



In [ ]:
# train_text_input_ids.shape

In [ ]:
# train_context_attention_mask.dtype

In [ ]:
# train_image_data_utterance.shape

In [ ]:
# train_text_input_ids.shape

In [ ]:
# train_text_attention_mask.shape

In [ ]:
# train_context_input_ids.shape

In [ ]:
# train_context_attention_mask.shape

In [ ]:
# train_audio_data_utterance.shape

In [ ]:
# train_image_data_utterance.shape

In [ ]:
# train_ground_truth.shape

In [ ]:
# valid_context_mask.shape

In [ ]:
# valid_context_id.shape

In [ ]:
# valid_truth.shape

In [ ]:
# test_audio_data_utterance.shape

In [ ]:
# test_context_input_ids.shape

In [ ]:
test_image_data.shape

torch.Size([69, 480, 2048])

In [ ]:
test_audio_data.shape


torch.Size([69, 1000, 768])

In [ ]:
test_text_id.shape

torch.Size([69, 500])

In [ ]:
test_text_mask.shape

torch.Size([69, 500])

In [ ]:
valid_ground_truth.shape

torch.Size([69])

In [ ]:
test_ground_truth.shape

torch.Size([69])

In [ ]:
train_audio_data_utterance.shape


torch.Size([552, 1000, 768])

In [ ]:
# train_loader = DataLoader(MultimodalSarcasmDataset(train_text_input_ids, train_text_attention_mask, train_context_input_ids, train_context_attention_mask,train_audio_broadcast_utterance, train_image_broadcast_utterance, train_ground_truth), batch_size=32, shuffle = True)
# valid_loader = DataLoader(MultimodalSarcasmDataset(valid_text_input_ids, valid_text_attention_mask, valid_context_input_ids, valid_context_attention_mask, valid_audio_broadcast_utterance, valid_image_broadcast_utterance, valid_ground_truth), batch_size = 32, shuffle = False)
# test_loader = DataLoader(MultimodalSarcasmDataset(test_text_input_ids, test_text_attention_mask, test_context_input_ids, test_context_attention_mask, test_audio_broadcast_utterance, test_image_broadcast_utterance, test_ground_truth), batch_size=32, shuffle = False)

# train_loader = DataLoader(MultimodalSarcasmDataset(train_text_input_ids, train_text_attention_mask, train_context_input_ids, train_context_attention_mask, train_audio_data_utterance, train_image_data_utterance, train_ground_truth), batch_size=32, shuffle = True)
# # valid_loader = DataLoader(MultimodalSarcasmDataset(valid_text_input_ids, valid_text_attention_mask, valid_context_input_ids, valid_context_attention_mask,   valid_audio_data, valid_image_data, valid_ground_truth), batch_size = 32, shuffle = False)
# test_loader = DataLoader(MultimodalSarcasmDataset(test_text_input_ids, test_text_attention_mask, test_context_input_ids, test_context_attention_mask,  test_audio_data_utterance1, test_image_data_utterance1, test_ground_truth), batch_size=32, shuffle = False)

train_loader = DataLoader(MultimodalSarcasmDataset(train_text_input_ids, train_text_attention_mask,  train_audio_data_utterance, train_image_data_utterance, train_ground_truth), batch_size=32, shuffle = True)
valid_loader = DataLoader(MultimodalSarcasmDataset(valid_text_input_ids, valid_text_attention_mask,    valid_audio_data, valid_image_data, valid_ground_truth), batch_size = 32, shuffle = False)
test_loader = DataLoader(MultimodalSarcasmDataset(test_text_id, test_text_mask,   test_audio_data, test_image_data, test_ground_truth), batch_size=32, shuffle = False)


print(test_loader)

# print(train_loader)

In [ ]:
len(train_loader.dataset)

552

In [ ]:
len(valid_loader.dataset)

69

In [ ]:
len(test_loader.dataset)

69

In [ ]:
# base_params_list = []
# new_params_list = []
# weight_decay = 1e-4
# for name, param in model.named_parameters():
#         if "classification_head" or "MAF_layer" in name:
#             new_params_list.append(param)
#         else:
#             base_params_list.append(param)
            
#         optimizer = torch.optim.AdamW(
#         [
#             {'params': base_params_list,'lr': 5e-6, 'weight_decay': weight_decay},
#             {'params': new_params_list,'lr': LEARNING_RATE, 'weight_decay': weight_decay}            
#         ],
#         lr=5e-6,
#         weight_decay=weight_decay
#     )

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
model.config

BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 1024,
  "model_ty

In [ ]:
DEVICE

device(type='cuda')

In [ ]:
model = model.to(DEVICE)
model


MultimodalBartForSequenceClassification(
  (model): MultimodalBartModel(
    (shared): Embedding(50267, 768)
    (encoder): MultiModalBartEncoder(
      (embed_tokens): Embedding(50267, 768)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05

In [ ]:
def train_epoch(model, data_loader):
      model.train()
      epoch_train_loss = 0.0
    
      
      for step, batch in enumerate(tqdm(data_loader, desc = 'Training Iteration')):
        # for i, t in enumerate(batch):
        #     print("Inside hello")
        #     print(i, " : ", type(t))
        batch = tuple(t.to(DEVICE) for t in batch)
        # input_ids, attention_mask, context_input_ids, context_attention_mask, acoustic_input, visual_input, labels = batch
        input_ids, attention_mask,  acoustic_input, visual_input, labels = batch
        optimizer.zero_grad()
        # print("Input ids shape : ", input_ids.shape)
        # print("Input ids shape : ", input_ids.shape)
        outputs = model(input_ids = input_ids,
                        attention_mask = attention_mask,
                        # context_input_ids = context_input_ids,
                        # context_attention_mask = context_attention_mask,
                        acoustic_input = acoustic_input,
                        visual_input = visual_input,
                        labels = labels)

        loss = outputs['loss']
        epoch_train_loss += loss.item()

        # print("Batch wise loss : ", epoch_train_loss)

        loss.backward()
        optimizer.step()

    

      print("Epoch train loss : ", epoch_train_loss)

In [ ]:
def valid_epoch(model, data_loader):
  model.eval()
  predictions = []
  gold = []

  valid_loss = 0.0
  with torch.no_grad():
    for step, batch in enumerate(tqdm(data_loader)):
      batch = tuple(t.to(DEVICE) for t in batch)
      # input_ids, attention_mask, context_input_ids, context_attention_mask, acoustic_input, visual_input, labels = batch
      input_ids, attention_mask,  acoustic_input, visual_input, labels = batch

      outputs = model(input_ids = input_ids,
                            attention_mask = attention_mask,
                            # context_input_ids = context_input_ids,
                            # context_attention_mask = context_attention_mask,
                            acoustic_input = acoustic_input,
                            visual_input = visual_input,
                            labels = labels)
      
      logits = outputs['logits']
      loss = outputs['loss']

      valid_loss += loss.item()



      pred = logits.argmax(dim = -1)

      predictions.extend(pred.tolist())
      gold.extend(labels.tolist())

  return valid_loss, predictions, gold



In [ ]:


def test_epoch(model, data_loader):
    model.eval()
    predictions = []
    gold = []

    correct = 0
    with torch.no_grad():
        for step, batch in enumerate(tqdm(data_loader)):
            batch = tuple(t.to(DEVICE) for t in batch)
            # input_ids, attention_mask, context_input_ids, context_attention_mask, acoustic_input, visual_input, labels = batch
            input_ids, attention_mask,  acoustic_input, visual_input, labels = batch

            outputs = model(input_ids = input_ids,
                            attention_mask = attention_mask,
                            # context_input_ids = context_input_ids,
                            # context_attention_mask = context_attention_mask,
                      
                            acoustic_input = acoustic_input,
                            visual_input = visual_input,
                            labels = labels)

            logits = outputs['logits']

            pred = logits.argmax(dim = -1)

            predictions.extend(pred.tolist())

            gold.extend(labels.tolist())

            correct += int((pred == labels).sum())

    return correct/len(data_loader.dataset), predictions, gold 

In [ ]:
class EarlyStopping:
  def __init__(self, patience, min_delta):
    self.patience = patience
    self.min_delta = min_delta
    self.counter = 0
    self.min_validation = np.inf

  def early_stop(self, valid_loss):
    if valid_loss < self.min_validation:
      self.min_validation = valid_loss
      self.counter = 0
    elif valid_loss > (self.min_validation + self.min_delta):
      self.counter += 1
      if self.counter >= self.patience:
        return True
    return False          

In [ ]:
early_stopper = EarlyStopping(patience = 15, min_delta = 0.2)
early_stopper

In [ ]:

def train_and_validation(model, train_loader, valid_loader):
  # lowest_loss = 1e6
  best_f1 = 0.0
  # min_loss = 1e6
  for epoch in range(30):
    print("\n=============Epoch : ", epoch)
    train_epoch(model, train_loader)
    valid_loss, valid_pred, valid_gold = valid_epoch(model, valid_loader)

    if early_stopper.early_stop(valid_loss):
      break

    print("Length of predictions : ", len(valid_pred))
    print("Length of gold : ", len(valid_gold))
    print("Valid loss : ", valid_loss)
    print("\n Valid Accuracy : ", accuracy_score(valid_gold, valid_pred))
    print("\n Valid Precision : ", precision_score(valid_gold, valid_pred, average = 'weighted'))
    print("\n Valid Recall : ", recall_score(valid_gold, valid_pred, average = 'weighted'))
    print("\nValid F1 score : ", f1_score(valid_gold, valid_pred, average = 'weighted')) 

    
    curr_f1 = f1_score(valid_gold, valid_pred, average = 'weighted')

    curr_loss = valid_loss
    # if((curr_f1 > best_f1) and (epoch>=4)):
    if(curr_f1 > best_f1):
    # if(curr_loss < min_loss):    
    # if(curr_loss < lowest_loss):    
      best_f1 = curr_f1
      # min_loss = curr_loss
      # print("Valid pred : ", valid_pred)
      # print('valid_gold : ', valid_gold)
      torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/32/saved_model/audio_video_fusion/best_model_epoch_'+str(epoch)+'_best_f1_'+str(int(best_f1*100))+'_foldNum_'+str(foldNum)+'.pt')
      print("model saved\n")




In [ ]:
# train_and_validation(model, train_loader, test_loader)
train_and_validation(model, train_loader, valid_loader)


=============Epoch :  0


Training Iteration: 100%|██████████| 18/18 [00:16<00:00,  1.09it/s]


Epoch train loss :  12.493129014968872


100%|██████████| 3/3 [00:00<00:00,  3.88it/s]


Length of predictions :  69
Length of gold :  69
Valid loss :  2.151444673538208

 Valid Accuracy :  0.5362318840579711

 Valid Precision :  0.2875446334803613

 Valid Recall :  0.5362318840579711

Valid F1 score :  0.3743505605687722
model saved


=============Epoch :  1


...

100%|██████████| 3/3 [00:00<00:00,  3.82it/s]


Length of predictions :  69
Length of gold :  69
Valid loss :  5.682573318481445

 Valid Accuracy :  0.7536231884057971

 Valid Precision :  0.7615795142969055

 Valid Recall :  0.7536231884057971

Valid F1 score :  0.7536231884057971

=============Epoch :  28


Training Iteration: 100%|██████████| 18/18 [00:13<00:00,  1.34it/s]


Epoch train loss :  0.03251035678840708


100%|██████████| 3/3 [00:00<00:00,  3.82it/s]


In [ ]:
# best_model_epoch_12_best_f1_72_foldNum_0.pt
# best_model_epoch_9_best_f1_69_foldNum_1.pt
# best_model_epoch_9_best_f1_79_foldNum_2.pt
# best_model_epoch_9_best_f1_73_foldNum_3.pt
# best_model_epoch_7_best_f1_75_foldNum_4.pt

In [ ]:
# path = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/stratify/best_model_epoch_11_best_f1_78_foldNum_0.pt'

path = '/content/drive/MyDrive/Colab Notebooks/32/saved_model/audio_video_fusion/best_model_epoch_15_best_f1_82_foldNum_0.pt'

In [ ]:
# PATH_0 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_12_f1_76.pt'
# PATH_1 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_13_f1_78_foldNum_1.pt'
# PATH_2 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_9_f1_64_foldNum_2.pt'
# PATH_3 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_5_f1_73_foldNum_3.pt'
# PATH_4 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_4_f1_74_foldNum_4.pt'

In [ ]:

# PATH_0 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_6_f1_50_foldNum_0.pt'
# PATH_1 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_4_f1_70_foldNum_1.pt'
# PATH_2 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_11_f1_63_foldNum_2.pt'
# PATH_3 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_4_f1_44_foldNum_3.pt'
# PATH_4 = '/content/drive/MyDrive/Colab Notebooks/32/saved_model_f1/best_model_epoch_4_f1_62_foldNum_4.pt'

In [ ]:
foldNum

0

In [ ]:
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:


acc, test_pred, test_gold = test_epoch(model, test_loader)

print(acc)

print("\nAccuracy : ", accuracy_score(test_gold, test_pred))
print("\nPrecision : ", precision_score(test_gold, test_pred, average = 'weighted'))
print("\nRecall : ", recall_score(test_gold, test_pred, average = 'weighted'))
print("\nF1 score : ", f1_score(test_gold, test_pred, average = 'weighted'))
        




print('\nConfusion Matrix : ', confusion_matrix(test_gold, test_pred))

sar = 0

for j in range(len(test_pred)):
  if((test_pred[j]==1) and (test_gold[j] == 1)):
    sar+=1


print("Sarcastic count : ", sar)


100%|██████████| 3/3 [00:00<00:00,  3.58it/s]

0.7101449275362319

Accuracy :  0.7101449275362319

Precision :  0.7105489067313033

Recall :  0.7101449275362319

F1 score :  0.7079152731326643

Confusion Matrix :  [[20 12]
 [ 8 29]]
Sarcastic count :  29


In [ ]:
# len(test_gold)

In [ ]:
# with open('/content/drive/MyDrive/Colab Notebooks/32/test_gold.p', 'wb') as f:
#   pickle.dump(test_gold, f)

In [ ]:
# with open('/content/drive/MyDrive/Colab Notebooks/32/test_pred.p', 'wb') as f:
#   pickle.dump(test_pred, f)

In [ ]:
# for j in range(len(test_pred)):
#   if((test_pred[j]==0) and (test_gold[j] == 1)):
#     print(j," : ", test_did[j])

In [ ]:
# for j in range(len(test_pred)):
#   if((test_pred[j]==1) and (test_gold[j] == 0)):
#     print(j," : ", test_did[j])

In [ ]:
# for j in range(len(test_pred)):
#   if((test_pred[j]==0) and (test_gold[j] == 0)):
#     print(j," : ", test_did[j])

In [ ]:
# for j in range(len(test_pred)):
#   if((test_pred[j]==1) and (test_gold[j] == 1)):
#     print(j," : ", test_did[j])

In [ ]:
# valid_loss, valid_pred, valid_gold = valid_epoch(model, valid_loader)

# # print(acc)

# print("\nAccuracy : ", accuracy_score(valid_gold, valid_pred))
# print("\nPrecision : ", precision_score(valid_gold, valid_pred, average = 'weighted'))
# print("\nRecall : ", recall_score(valid_gold, valid_pred, average = 'weighted'))
# print("\nF1 score : ", f1_score(valid_gold, valid_pred, average = 'weighted'))

In [ ]:
# valid_loss, valid_pred, valid_gold = valid_epoch(model, valid_loader)

# # print(acc)

# print("\nAccuracy : ", accuracy_score(valid_gold, valid_pred))
# print("\nPrecision : ", precision_score(valid_gold, valid_pred))
# print("\nRecall : ", recall_score(valid_gold, valid_pred))
# print("\nF1 score : ", f1_score(valid_gold, valid_pred))

In [ ]:
# valid_loss, valid_pred, valid_gold = valid_epoch(model, valid_loader)

# # print(acc)

# print("\nAccuracy : ", accuracy_score(valid_gold, valid_pred))
# print("\nPrecision : ", precision_score(valid_gold, valid_pred, average = 'micro'))
# print("\nRecall : ", recall_score(valid_gold, valid_pred, average = 'micro'))
# print("\nF1 score : ", f1_score(valid_gold, valid_pred, average = 'micro'))

In [ ]:


# acc, test_pred, test_gold = test_epoch(model, test_loader)

# # print(acc)

# print("\nAccuracy : ", accuracy_score(test_gold, test_pred))
# print("\nPrecision : ", precision_score(test_gold, test_pred))
# print("\nRecall : ", recall_score(test_gold, test_pred))
# print("\nF1 score : ", f1_score(test_gold, test_pred))
        







                                                                 






In [ ]:


# acc, test_pred, test_gold = test_epoch(model, test_loader)

# # print(acc)

# print("\nAccuracy : ", accuracy_score(test_gold, test_pred))
# print("\nPrecision : ", precision_score(test_gold, test_pred, average = 'micro'))
# print("\nRecall : ", recall_score(test_gold, test_pred, average = 'micro'))
# print("\nF1 score : ", f1_score(test_gold, test_pred, average = 'micro'))
        







                                                                 






In [ ]:
# test_pred

In [ ]:
# test_gold